# OpenEO Connection to EURAC Backend

In [1]:
import openeo
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# Define constants

# Connection
EURAC_DRIVER_URL = "http://saocompute.eurac.edu/openEO_0_3_0/openeo"
OUTPUT_FILE = "/tmp/openeo-wcps.nc"
OUTFORMAT = "netcdf"

user = "group1"
password = "test123"

# Data
PRODUCT_ID = "S2_L2A_T32TPS_20M"

DATE_START = "2016-01-01"
DATE_END = "2016-03-10"

IMAGE_WEST = 652000
IMAGE_EAST = 672000
IMAGE_NORTH = 5161000
IMAGE_SOUTH = 5181000
IMAGE_SRS = "EPSG:32632"

# Processes
NDVI_RED = "B04"
NDVI_NIR = "B8A"

In [3]:
# Connect with EURAC backend
connection = openeo.connect(EURAC_DRIVER_URL, auth_options={"username": user, "password": password})
connection

In [4]:
# Get available processes from the back end.
processes = connection.list_processes()
processes

[{'description': 'Creates a subset of data including only the values inside a given data range.',
  'exceptions': {},
  'name': 'filter_daterange',
  'parameters': {'extent': {'description': 'Temporal extent specified by a start and an end time, each formatted as a [RFC 3339](https://www.ietf.org/rfc/rfc3339) date-time. Open date ranges are supported and can be specified by setting one of the times to null. Setting both entries to null is not allowed.',
    'required': True,
    'schema': {'example': ['2016-01-01T00:00:00Z', '2017-10-01T00:00:00Z'],
     'format': 'temporal_extent',
     'items': {'format': 'date-time', 'type': ['string', 'null']},
     'maxItems': 2,
     'minItems': 2,
     'type': 'array'}},
   'imagery': {'description': 'EO data to process.',
    'required': True,
    'schema': {'format': 'eodata', 'type': 'object'}}},
  'returns': {'description': 'Processed EO data.',
   'schema': {'format': 'eodata', 'type': 'object'}},
  'summary': 'Filters by Temporal Extent'},

In [5]:
# Retrieve the list of available collections
collections = connection.list_collections()

list(collections)[:2]

[{'description': 'openEO_S2_32632_60m_L2A',
  'extent': {'spatial': [7.708999870256598,
    45.958258688421424,
    11.802846682908976,
    47.819474402432704],
   'temporal': ['2018-06-04T00:00:00.000Z', '2018-06-23T00:00:00.000Z']},
  'license': 'CC-BY-4.0',
  'links': [{'href': 'http://saocompute.eurac.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=DescribeCoverage&COVERAGEID=openEO_S2_32632_60m_L2A',
    'rel': 'self'},
   {'href': 'https://creativecommons.org/licenses/by/4.0/', 'rel': 'license'}],
  'name': 'openEO_S2_32632_60m_L2A',
  'title': 'openEO_S2_32632_60m_L2A'},
 {'description': 'openEO_S2_32631_10m_L2A',
  'extent': {'spatial': [4.436120625114215,
    51.35443938977266,
    6.141769904458253,
    53.208201372165696],
   'temporal': ['2018-04-18T00:00:00.000Z', '2018-07-12T00:00:00.000Z']},
  'license': 'CC-BY-4.0',
  'links': [{'href': 'http://saocompute.eurac.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=DescribeCoverage&COVERAGEID=openEO_S2_32631_10m_L2A',


In [6]:
# Get detailed information about a collection
process = connection.describe_collection(PRODUCT_ID)
process

{'description': 'S2_L2A_T32TPS_20M',
 'eo:bands': {'AOT': {'common_name': 'AOT',
   'offset': 'AOT',
   'resolution': 'AOT',
   'scale': 'AOT'},
  'B02': {'common_name': 'B02',
   'offset': 'B02',
   'resolution': 'B02',
   'scale': 'B02'},
  'B03': {'common_name': 'B03',
   'offset': 'B03',
   'resolution': 'B03',
   'scale': 'B03'},
  'B04': {'common_name': 'B04',
   'offset': 'B04',
   'resolution': 'B04',
   'scale': 'B04'},
  'B05': {'common_name': 'B05',
   'offset': 'B05',
   'resolution': 'B05',
   'scale': 'B05'},
  'B06': {'common_name': 'B06',
   'offset': 'B06',
   'resolution': 'B06',
   'scale': 'B06'},
  'B07': {'common_name': 'B07',
   'offset': 'B07',
   'resolution': 'B07',
   'scale': 'B07'},
  'B11': {'common_name': 'B11',
   'offset': 'B11',
   'resolution': 'B11',
   'scale': 'B11'},
  'B12': {'common_name': 'B12',
   'offset': 'B12',
   'resolution': 'B12',
   'scale': 'B12'},
  'B8A': {'common_name': 'B8A',
   'offset': 'B8A',
   'resolution': 'B8A',
   'scale':

In [7]:
# Select collection product
datacube = connection.imagecollection(PRODUCT_ID)

print(datacube.to_json())

{'name': 'S2_L2A_T32TPS_20M', 'process_id': 'get_collection'}

In [8]:
# Specifying the date range and the bounding box
datacube = datacube.filter_bbox(west=IMAGE_WEST, east=IMAGE_EAST, north=IMAGE_NORTH,
                                             south=IMAGE_SOUTH, crs=IMAGE_SRS)
datacube = datacube.filter_daterange(extent=[DATE_START, DATE_END])

print(datacube.to_json())

{'extent': ['2016-01-01', '2016-03-10'],
 'imagery': {'extent': {'crs': 'EPSG:32632',
   'east': 672000,
   'north': 5161000,
   'south': 5181000,
   'west': 652000},
  'imagery': {'name': 'S2_L2A_T32TPS_20M', 'process_id': 'get_collection'},
  'process_id': 'filter_bbox'},
 'process_id': 'filter_daterange'}

In [9]:
# Sending the job to the backend
job = datacube.create_job()
job.start_job()
job

In [16]:
# Describe Job
job.describe_job()

{'consumed_credits': None,
 'job_id': 'b6fb84ad-056e-449a-91e9-56d2515aa5b2',
 'output': '{"format":null,"parameters":{}}',
 'process_graph': '{"extent":["2016-01-01","2016-03-10"],"process_id":"filter_daterange","imagery":{"extent":{"east":672000,"crs":"EPSG:32632","south":5181000,"north":5161000,"west":652000},"process_id":"filter_bbox","imagery":{"process_id":"get_collection","name":"S2_L2A_T32TPS_20M"}}}',
 'status': 'QUEUED',
 'submitted': '21 May 2019 13:04:24 GMT',
 'updated': '21 May 2019 13:04:24 GMT',
 'user_id': None}

In [19]:
# Download job result
resp = datacube.download(OUTPUT_FILE, OUTFORMAT)
#job.download_results(OUTPUT_FILE)

resp

ConnectionAbortedError: <html><head><title>Apache Tomcat/7.0.76 - Error report</title><style><!--H1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} H2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} H3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} BODY {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} B {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} P {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;}A {color : black;}A.name {color : black;}HR {color : #525D76;}--></style> </head><body><h1>HTTP Status 500 - org.json.JSONException: A JSONObject text must begin with '{' at 1 [character 2 line 1]</h1><HR size="1" noshade="noshade"><p><b>type</b> Exception report</p><p><b>message</b> <u>org.json.JSONException: A JSONObject text must begin with '{' at 1 [character 2 line 1]</u><

In [7]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np

# open a local NetCDF file or remote OPeNDAP URL
url = '/tmp/openeo-wcps.nc'
nc = netCDF4.Dataset(url)

# examine the variables
print (nc.variables.keys())
print (nc.variables['B8A'])

# sample every 10th point of the 'B8A' variable
topo = nc.variables['B8A'][::1,::1]
topo = np.flipud(np.rot90(topo)) 

# make image
plt.figure(figsize=(100,100))
plt.imshow(topo,origin='lower') 
plt.title('EURAC Plot')
plt.savefig('/tmp/image.png', bbox_inches=0)

odict_keys(['B8A', 'E', 'N'])
<class 'netCDF4._netCDF4.Variable'>
float64 B8A(E, N)
    missing_value: NaN
    units: 10^0
unlimited dimensions: 
current shape = (1000, 1000)
filling off



/data/.pyenv/versions/miniconda3-latest/envs/openeo-client/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  
